In [ ]:
import requests
import pyspark.sql.functions as sf

## First preparation (streams)

In [ ]:
# All POMS data
poms = spark.read.parquet("gs://mit-processed-events-prod.npo-data.nl/poms-enriched/")
print("Total poms: ", poms.count())

# Filter on broadcast and availability of locations
poms = poms.filter((sf.col("type")=="BROADCAST") & (sf.size("locations") > 0))
print("Filtered poms: ", poms.count())

# Select the possible used features
poms = poms.select([c for c in poms.columns if c in ['age_rating', 'broadcaster', 'credits', 'descriptions', 'genres', 'mid', 'locations', 'sortDate', 'titles']])
poms.printSchema()

In [ ]:
def locations(df):
    stream = []
    for index, row in df.iterrows():
        stream_bool = None
        for location in row['locations']:
            if (location.program_url.startswith("npo+drm://") or location.program_url.startswith("npo://")) & (location.platform == "INTERNETVOD" or location.platform == "PLUSVOD") & (location.publish_start != '0' or location.publish_stop != '0'):
                stream_bool = True
                break
        stream.append(stream_bool)
    df['stream'] = stream

locations(df)
df = df.dropna()

In [ ]:
poms2 = spark.createDataFrame(df)
poms2 = poms2.drop('stream')
poms2 = poms2.drop('locations')
poms2.write.parquet("gs://dataproc-jupyter-eileen.npo-data.nl/data/poms_data")

## Further preparation

In [ ]:
poms = spark.read.parquet("gs://dataproc-jupyter-eileen.npo-data.nl/data/poms_data/")
print(poms.count())

In [ ]:
df = poms.toPandas()

In [ ]:
def descriptions(df):
    descriptions = []
    for index, row in df.iterrows():
        descript = None
        for description in row['descriptions']:
            if (description.type == "MAIN"):
                descript = description.value
            elif (description.type == "SHORT"):
                descript = description.value
            elif (description.type == "KICKER"):
                descript = description.value
            else:
                continue
        descriptions.append(descript)
    df['description'] = descriptions

def titles(df):
    titles = []
    for index, row in df.iterrows():
        title = None
        for title in row['titles']:
            if (title.type == "MAIN"):
                title = title.value
                break;
            else:
                continue
        titles.append(title)
    df['title'] = titles

descriptions(df)
titles(df)
df = df.drop(columns='descriptions')
df = df.drop(columns='titles')

In [ ]:
poms2 = spark.createDataFrame(df)

def replace_age(column, value):
    return sf.when(column != value, column).otherwise(sf.lit("ALL"))

poms2 = poms2.withColumn("age_rating", replace_age(sf.col("age_rating"), ""))
poms2.write.parquet("gs://dataproc-jupyter-eileen.npo-data.nl/data/poms_data_v2")

## Preparation subtitles (with partitioning)

In [ ]:
poms = spark.read.parquet("gs://dataproc-jupyter-eileen.npo-data.nl/data/poms_data_v2/")
poms.printSchema()

In [ ]:
poms.count()

In [ ]:
poms = poms.repartition(25)
poms.write.parquet("gs://dataproc-jupyter-eileen.npo-data.nl/data/poms_partition25/")

In [ ]:
# run subtitle on range partitions
def sub_extract(df):
    subtitles = []
    poms_endpoint = "https://rs.poms.omroep.nl/v1/api/subtitles/"
    for index, row in df.iterrows():
        mid = row['mid']
        r = requests.get(poms_endpoint + mid + "/nl_NL/CAPTION.vtt")
        sub = r.text.encode('ascii','ignore') # encode subtitles
        sub = sub.lower().split('\n\n') # lower and split
        sub = sub[1:] # remove first entry of subtitles 'webvtt'
        sub = [line.split('\n', 2)[-1].replace('\n', ' ') for line in sub] # remove display time info and '\n' in subtitle text
        sub = u" ".join(sub)
        subtitles.append(sub)
    df['sub'] = subtitles

for i in range(0,25):
    print(i)
    poms_part = spark.read.parquet("gs://dataproc-jupyter-eileen.npo-data.nl/data/poms_partition25/part-000" 
                                   + str(i) +"-cf0443fe-b3d1-4798-866a-b044e1e85628-c000.snappy.parquet")
    df = poms_part.toPandas()
    sub_extract(df)
    poms_part2 = spark.createDataFrame(df)
    poms_part2.write.parquet("gs://dataproc-jupyter-eileen.npo-data.nl/data/sub_partition25/" + str(i))

In [ ]:
paths = []
for i in range (0, 25):
    paths.append('gs://dataproc-jupyter-eileen.npo-data.nl/data/sub_partition25/' + str(i))

poms = spark.read.parquet(*paths)
poms.write.parquet("gs://dataproc-jupyter-eileen.npo-data.nl/data/poms_sub/")